**In this experiment we will study the following problem:  a lot of really good algorithms for matrix completion problem, that have a good quality of prediction, measured by RMSE metrics, have a poor quality of prediction, measured by special "recommender metrics." We deside to analyse this quetsion and compare the quality of our algorithms, measured by different metrics.**

**There by $HR_{10}$ and $HR_{20}$ we denote Hit Rate for top-10 and top-20 recommendations respectively.**

**For estimating quality of our algoritms we will use 5-fold cross validation. But, to reduce compatational time we will choose best params, using not so big grid of parameters and will find the best pair, using only one fold. In general, of course, we sholud do a more exaustive grid search and use "full" cross-validation procedure. But in practice, expert articles said that the result obtained will also be good.**

**We can tune approximation rank and regularization coefficient for FastALS and ALS respectively and approximation rank for Riemannian Optimization algorithm.**

# Data_Preparation

In [1]:
# Libraries and provided functions
from sklearn.model_selection import KFold
from sklearn.model_selection import ParameterGrid
from sklearn.utils import shuffle
import pandas as pd
import zipfile
import wget
from io import StringIO 
import numpy as np
import scipy as sp
from scipy import sparse
from  scipy.sparse import linalg
from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline

from collections import namedtuple
import sys

def get_movielens_1M_data(local_file=None):
    '''Downloads movielens data and create Pandas DataFrame
    '''
    if not local_file:
        print('Downloading data...')
        zip_file_url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'
        zip_contents = wget.download(zip_file_url)
        print('Done.')
    else:
        zip_contents = local_file
    
    print('Loading data into memory...')
    with zipfile.ZipFile(zip_contents) as zfile:
        zdata = zfile.read('ml-1m/ratings.dat').decode()
        delimiter = ';'
        zdata = zdata.replace('::', delimiter) # makes data compatible with pandas c-engine
        ml_data = pd.read_csv(StringIO(zdata), sep=delimiter, header=None, engine='c',
                                  names=['userid', 'movieid', 'rating', 'timestamp'],
                                  usecols=['userid', 'movieid', 'rating'])
    
    
    
    print('Done.')
    return ml_data

In [2]:
def make_sparse(ml_data, dtype=np.int64):
    """
    Normalize data and make sparse matrix from pandas DataFrame
    Args:
        ml_data: (pandas DataFrame): Data Frame of users and ratings
    Returns:
        datamatrix (scipy.sparse): Sparse matrix (users - items).
    
    """
    
    # normalize indices to avoid gaps
    ml_data['movieid'] = ml_data.groupby('movieid', sort=False).grouper.group_info[0]
    ml_data['userid'] = ml_data.groupby('userid', sort=False).grouper.group_info[0]
    
    # build sparse user-movie matrix
    data_shape = ml_data[['userid', 'movieid']].max() + 1
    data_matrix = sp.sparse.csr_matrix((ml_data['rating'],
                                       (ml_data['userid'], ml_data['movieid'])),
                                        shape=data_shape, dtype=dtype)
    
    return data_matrix

In [3]:
def data_preproc(data, mode = 'hr', random_state=123):
    
    """
    Args:
        data (pandas DataFrame): data frame of initial data
        mode (str): 'hr' or 'rmse'.
    Returns:
        ans (csr_matrix): csr_matrix of the data. 
        
    if mode == 'hr', then all ratings with 4 and 5 goes to 1, else 0.
    if mode == 'rmse', then ratings don't change.
    """
    data_shuffle = shuffle(data, random_state=random_state)
    
    if (mode == 'hr'):
        ans = data_shuffle[data_shuffle.rating >= 4].copy()
        #ans.rating = 1
        
        ans.reset_index(drop=True, inplace=True)
    
    elif mode == 'rmse':
        ans = data_shuffle.reset_index(drop=True, inplace=False)
        
    else:
        return ("Mistake. Please check input")
    
    ans = make_sparse(ans)
    
    
    return ans

In [4]:
data = get_movielens_1M_data(local_file='ml-1m.zip')

Loading data into memory...
Done.


In [5]:
data.head()

,userid,movieid,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [6]:
data_sparse = data_preproc(data, mode='hr', random_state=123)
data_sparse

<6038x3533 sparse matrix of type '<class 'numpy.int64'>'
	with 575281 stored elements in Compressed Sparse Row format>

In [7]:
data_array = data_sparse.toarray()

# Functions for working with data:

In [8]:
def generate_hold_out(Arr, mode='hr', random_state=123, rmse_num = 3):
    """
    Geberates hold out set for recommender problem:
    Args:
        Arr (np.array): "Test" array from which we will select items.
        mode (str): 'hr' or 'rmse'. If 'hr' then get the top items for hold_out set. If 'rmse', 
        then perform random selection for hold_out set
        random_state (int): random_state for reproducible results.
        rmse_num (int): Number of random choised elements for hold_out set if mode='rmse'
    Returns
        hold_out (np.array): indices of hold_out elements. 
    """
    np.random.seed(random_state)
    if mode == 'hr':
        Arg_Arr = np.argsort(-Arr)
        hold_out = Arg_Arr[:, 0]
    elif mode == 'rmse':
        hold_out = np.zeros((Arr.shape[0], rmse_num))
        
        for it, user in enumerate(Arr):
            
            hold_out[it] = np.random.choice(a = Arr.shape[1], size=rmse_num, replace=False)
    
    else:
        return "Mistake, check input!"
    
    return hold_out[:, None]

In [9]:
def get_train_test(Arr, num):
    """
    Get train and test sets
    Args:
        Arr (np.array): data_array
        num (int): number of first elements to take
    Returns:
        X_train (no.array): train set
        X_test (np.array): test set
    """

    X_train = data_array[:num]
    X_test = data_array[num:]
    
    return (X_train, X_test)

In [10]:
def change_train_test(X_train, X_test, hold_out):
    """
    Check if the hold_out item does not lie in train set. If so, delete this user and assygn it to train set.
    Args:
        X_train (np.array): train set
        X_test (np.array): test set
        hold_out (np.array): hold_out indices
    Returns:
        X_train_copy (np.array): Changed copy of X_train in which we added test users, for which we find items,
        that do not liy in training set
        
        X_test_copy (np.array): Changed copy of X_test from which we delete test users, for which we find items,
        that do not liy in training set
        
        hold_out_copy (np.array): Changed copy of hold_out from which we delete rows, that corresponding 
        to test users, for which we find items, that do not liy in training set
        
    """
    X_train_copy = X_train.copy()
    X_test_copy = X_test.copy()
    hold_out_copy = hold_out.copy()
    
    counter = 0
    for it, user in enumerate(X_test):
        if np.sum(X_train[:, hold_out[it]]) == 0:
            X_test_copy = np.delete(X_test, it, 0)
            hold_out_copy = np.delete(hold_out, it, 0)
            X_train_copy = np.vstack([X_train_copy, user])
            counter += 1
        else:
            pass
    
    return (X_train_copy, X_test_copy, hold_out_copy, counter)

In [11]:
def get_recommender_problem(train, test, hold_out):
    """
    Zero hold_out elements in the test and create sparse matrix
    Args:
        train (np.array): train set
        test (np.array): test set
        hold_out (np.array): hold_out indices
        
    Returns:
        problem (sparse.csr_matrix) full data matrix with zerod hold_out elements
    """
    ind = np.arange(test.shape[0])[:, None]
    
    test_new = test.copy()
    test_new[ind, hold_out] = 0
    
    train_sparse = sparse.csr_matrix(train)
    test_sparse = sparse.csr_matrix(test_new)
    
    problem = sparse.vstack([train_sparse, test_sparse])
    
    return problem

# Metrics:

In [12]:
def calculate_HR(X_test_pred, hold_out, top_k = 10):
    """
    Calculate Hit Rate score. 
    Args:
        X_test_pred (np.array): Predictions
        hold_out (np.array): hold_out indices
        top_k (int): number of top elements to choose for evaluating
    Returns:
        total_score (int): number of hits 
        total_score/N (float): hit rate
    """
    total_score = 0
    
    for it, user in enumerate(X_test_pred):

        user_argsort = np.argsort(-user)
        total_score += np.intersect1d(hold_out[it], user_argsort[:top_k]).shape[0]

    
    return total_score, total_score/X_test_pred.shape[0]

In [13]:
def calculate_RMSE(prediction, true, hold_out):
    
    """
    Calculate RMSE score. 
    Args:
        prediction (np.array): Predictions
        true (np.array): true labels
        hold_out (np.array): hold_out indices
    Returns:
        ans (float): RMSE score
    """
    
    ind = np.arange(true.shape[0])[:, None]   
    
    diff = prediction[ind, hold_out] - true[ind, hold_out]
    ans = np.sqrt(np.mean(diff**2))
    
    return (ans)

In [14]:
import numpy as np
import scipy.sparse as sparse
from time import time

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('whitegrid')

from Algorithms import FastALS
from Algorithms import RiemannianOptimization
from Algorithms import ALS

from tqdm import tqdm

# Grid_Search_1_fold

In [15]:
num = int(data_array.shape[0]*0.8)
X_train, X_test = get_train_test(data_array, num)
    
hold_out = generate_hold_out(X_test, mode='hr', random_state=123)
(X_train, X_test, hold_out, counter) = change_train_test(X_train, X_test, hold_out)
problem = get_recommender_problem(X_train, X_test, hold_out)

In [16]:
def GridSearch_1_fold(estimator, params, problem, X_test, hold_out, metric, top_k = None, debug_mode = False):
    """
    Perfoms exhaustive grid search for parameters. 
    Find best parameters and calculte scores for the best parameters.
    
    Args:
        estimator: estimator to fit
        params (dic) dictionary of parameters
        problem (csr_matrix): data with zerod hold_out set
        X_test (np.array): test set
        hold_out (np.array): hold_out indices
        metric (str): 'hr' or 'rmse'.
        top_k (int): top elements using for evaluating score.
        debug_mode (bool): if True, print params and score on each iteration.
    Returns:
        best_score (float): best csore, obtained on best params
        best_params (dic): best parameters, that was found by procedure
    """
    
    
    param_grid = list(ParameterGrid(params))
    
    best_params = []
    
    if metric == 'hr':
        best_score = 0
    
    elif metric == 'rmse':
        best_score = np.inf
    
    else:
        return ("Check metric.")
    
    for param in param_grid:
        try:
            rank = param['rank']
            estimator._rank = rank
        except KeyError:
            pass
            
        try:
            max_iter = param['max_iter']
            estimator._max_iter = max_iter
        except KeyError:
            pass
        
        try:
            tol = param['tol']
            estimator._tol = tol
        except KeyError:
            pass
         
        try:
            reg_coef = param['reg_coef']
            estimator._reg_coef = reg_coef
        except KeyError:
            pass    
            
        
        estimator.fit(problem, trace=False, debug_mode=False)
        
        Pred = estimator.predict()
        X_pred = Pred[X_train.shape[0]:]
        
        if metric == 'hr':
            hits, cur_score = calculate_HR(X_pred, hold_out, top_k=top_k)
            
            if cur_score > best_score:
                best_score = cur_score
                best_params = param
        
        elif metric == 'rmse':
            cur_score = calculate_RMSE(X_pred, X_test, hold_out)
            
            if cur_score < best_score:
                best_score = cur_score
                best_params = param
            
        else:
            return("Check metric")
        
        if cur_score < best_score:
            best_score = cur_score
            best_params = param
        
        if debug_mode:
            print("rank: {0}, reg_coef: {1}, score: {2}".format(param['rank'], param['reg_coef'], cur_score))
                  
    return (best_score, best_params)

# Grid Search for FastALS

# Grid Search for FastALS

In [17]:
fast_als = FastALS.FastALS()

In [18]:
param_dic = {'rank': np.linspace(5, 200, 20, dtype=int).tolist(), 
             'reg_coef': np.linspace(1, 100, 20, dtype=float).tolist(),
             'max_iter': [1000], 'tol': [1e-5]}

In [19]:
fast_best_score_top_10, fast_best_params_top_10 = GridSearch_1_fold(fast_als, param_dic, problem, 
                                            X_test, hold_out, metric='hr', top_k = 10, debug_mode = False)

In [20]:
fast_best_score_top_20, fast_best_params_top_20 = GridSearch_1_fold(fast_als, param_dic, problem, 
                                            X_test, hold_out, metric='hr', top_k = 20, debug_mode = False)

# Grid Search for Riman

In [21]:
riman = RiemannianOptimization.RiemannianOptimization()

In [22]:
riman_best_score_top_10, riman_best_params_top_10 = GridSearch_1_fold(riman, param_dic, problem, 
                                            X_test, hold_out, metric='hr', top_k = 10, debug_mode = False)

In [23]:
riman_best_score_top_20, riman_best_params_top_20 = GridSearch_1_fold(riman, param_dic, problem, 
                                            X_test, hold_out, metric='hr', top_k = 20, debug_mode = False)

# Cross-validation

In [24]:
def cross_validation(cv, estimator, data_array, mode='hr', top_k = 10, random_state=123, debug_mode=False):
    it = 1

    np.random.seed(random_state)
    data_array_shuffle = shuffle(data_array)

    scores = []

    for train_index, test_index in cv.split(data_array_shuffle):
        X_train, X_test = data_array_shuffle[train_index], data_array_shuffle[test_index]
    
        hold_out = generate_hold_out(X_test, mode=mode, random_state=123*(it+1))
        (X_train, X_test, hold_out, counter) = change_train_test(X_train, X_test, hold_out)
        problem = get_recommender_problem(X_train, X_test, hold_out)
    
        #fast_als = FastALS.FastALS(rank=approx_rank, max_iter=max_iter, tol=tol, reg_coef=reg_coef)
        estimator.fit(problem, trace=False, debug_mode=False)
    
        Pred = estimator.predict()
        X_pred = Pred[X_train.shape[0]:]
        
        if mode == 'hr':
            cur_score = calculate_HR(X_pred, hold_out, top_k=top_k)
        elif mode == 'rmse':
            cur_score = calculate_RMSE(X_pred, X_test, hold_out)
        
        if debug_mode == True:
            print("It:", it, "cur_score:", cur_score)
    
        scores.append(cur_score)
    
    it += 1
    
    return (scores)

In [25]:
kf = KFold(n_splits=5)

**Cross_val for FastALS top10**

In [26]:
top_k = 10

max_iter = 2000
tol = 1e-5
approx_rank = fast_best_params_top_10['rank']
reg_coef = fast_best_params_top_10['reg_coef']

In [27]:
estimator = FastALS.FastALS(max_iter=max_iter, tol=tol, rank=approx_rank, reg_coef=reg_coef)
fast_scores_top_10 = cross_validation(kf, estimator, data_array, top_k=top_k)

**Cross_val for FastALS top10**

In [28]:
top_k = 20

max_iter = 2000
tol = 1e-5
approx_rank = fast_best_params_top_20['rank']
reg_coef = fast_best_params_top_20['reg_coef']

In [29]:
estimator = FastALS.FastALS(max_iter=max_iter, tol=tol, rank=approx_rank, reg_coef=reg_coef)
fast_scores_top_20 = cross_validation(kf, estimator, data_array, top_k = top_k)

**Cross_val for Riman top10**

In [30]:
top_k = 20

max_iter = 2000
tol = 1e-5
approx_rank = riman_best_params_top_10['rank']

In [31]:
estimator = RiemannianOptimization.RiemannianOptimization(max_iter=max_iter, tol=tol, 
                                                          rank=approx_rank, reg_coef=reg_coef)
riman_scores_top_10 = cross_validation(kf, estimator, data_array, top_k = top_k)

In [32]:
top_k = 20

max_iter = 2000
tol = 1e-5
approx_rank = riman_best_params_top_20['rank']

In [33]:
estimator = RiemannianOptimization.RiemannianOptimization(max_iter=max_iter, tol=tol, 
                                                          rank=approx_rank, reg_coef=reg_coef)
riman_scores_top_20 = cross_validation(kf, estimator, data_array, top_k = top_k)

In [34]:
fast_als_benchmarks = pd.DataFrame({"HR_10": [np.mean(fast_scores_top_10), np.std(fast_scores_top_10)], 
                               "HR_20": [np.mean(fast_scores_top_20), np.mean(fast_scores_top_20)]}, 
                                   index = ['mean', 'std'])

In [35]:
riman_benchmarks = pd.DataFrame({"HR_10": [np.mean(riman_scores_top_10), np.std(riman_scores_top_10)], 
                               "HR_20": [np.mean(riman_scores_top_20), np.mean(riman_scores_top_20)]}, 
                                   index = ['mean', 'std'])